# Cleaned BLP data from Qualtrics

This notebook will take in the raw csv exported from Qualtrics and will output a cleaned csv file with sociodemographic information.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/qualtrics.csv", header=0)
df.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q93_Type,Q95,Q96,Q122,Q122_4_TEXT,Q166,Q167,Q168,Q169,Q169_6_TEXT
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,Below is a consent form. Please read it and si...,"In the textbox below, please provide the serie...",Please write the email at which you would like...,What kind of Amazon gift card would you like t...,What kind of Amazon gift card would you like t...,1. Age,2. Gender,"3. Current place of residence (State/Province,...",4. Highest level of education achieved - Selec...,4. Highest level of education achieved - Other...
1,"{""ImportId"":""startDate"",""timeZone"":""America/Lo...","{""ImportId"":""endDate"",""timeZone"":""America/Los_...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID93_FILE_TYPE""}","{""ImportId"":""QID95_TEXT""}","{""ImportId"":""QID96_TEXT""}","{""ImportId"":""QID200""}","{""ImportId"":""QID200_4_TEXT""}","{""ImportId"":""QID166_TEXT""}","{""ImportId"":""QID167""}","{""ImportId"":""QID168_TEXT""}","{""ImportId"":""QID169""}","{""ImportId"":""QID169_6_TEXT""}"
2,2022-12-12 16:13:16,2022-12-12 16:25:12,IP Address,79.157.225.1,100,716,True,2022-12-12 16:25:13,R_1IQPnZyzEWxzNWU,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-20 22:32:09,2023-01-20 22:34:49,IP Address,71.143.195.14,100,159,True,2023-01-20 22:34:52,R_12RVfAO1DLjA01T,NaN,...,image/png,Anm099,marthajack@gmail.com,NaN,NaN,69,Female,California,College,NaN
4,2023-01-24 17:15:40,2023-01-24 17:21:11,IP Address,74.84.13.116,100,330,True,2023-01-24 17:21:12,R_6zjRGPqZHCoPfwt,NaN,...,image/png,acj047,latarver1@gmail.com,NaN,NaN,61,Female,Texas,Doctorate,NaN


Keep only rows where the value for 'Finished' is either 'Finished' or 'True', i.e., where the participant went as far as Qualtrics allowed them to.

In [3]:
print(df.shape)
df = df[(df['Finished']=='Finished') | (df['Finished']=='True')].copy()
df.shape

(307, 257)


(169, 257)

Merge together columns with the same name. The survey was in three languages so this removes redundancies.

In [4]:
# Q16 is repeated, fix this
df.rename(columns={'Q16_1.1': 'Q16_5_1', 'Q16_2.1': 'Q16_5_2', 'Q16_3.1': 'Q16_5_3',
                  'Q16_1.2': 'Q16_1','Q16_2.2': 'Q16_2', 'Q16_3.2': 'Q16_3',
                  'Q16_1.3': 'Q16_5_1','Q16_2.3': 'Q16_5_2', 'Q16_3.3': 'Q16_5_3',
                  'Q16_1.4': 'Q16_1', 'Q16_2.4': 'Q16_2', 'Q16_3.4': 'Q16_3',
                  'Q16_1.5': 'Q16_5_1','Q16_2.5': 'Q16_5_2', 'Q16_3.5': 'Q16_5_3'},
          inplace = True)

In [5]:
# first fix duplicate names
def add_duplicates(x):
    if "." in x:
        return x[:-2]
    else:
        return x
df.columns = df.columns.to_series().apply(add_duplicates)

In [6]:
# rename some duplicate columns
df.rename(columns={'Q166': 'Age', 'Q95': 'Q33', 'Q167': 'Gender', 'Q168': 'Residence',
                   'Q169': 'Education', 'Q169_6_TEXT': 'Education_6_TEXT', 'Q34': 'Q33',
                  'Q96': 'email'},
          inplace = True)

In [7]:
#define function to merge columns with same names together
def same_merge(x):
    return ''.join(x[x.notnull()].astype(str))

#define new DataFrame that merges columns with same names together
df_new = df.groupby(level=0, axis=1).apply(lambda x: x.apply(same_merge, axis=1))
print(df_new.shape)
df_new.head()

(169, 104)


,Age,Amazon_cat,Amazon_eng,Amazon_spa,Consent_intro_Id,Consent_intro_Name,Consent_intro_Size,Consent_intro_Type,DistributionChannel,Duration (in seconds),...,RecipientLastName,RecordedDate,Residence,ResponseId,StartDate,Status,UserLanguage,email,lang_pref,prof
0,1. Age1. EdadEdat1. Age,D'on prefereix rebre el pagament electrònic d'...,From where would you prefer to receive your el...,¿De dónde prefiere recibir el pago electrónico...,Below is a consent form. Please read it and si...,Below is a consent form. Please read it and si...,Below is a consent form. Please read it and si...,Below is a consent form. Please read it and si...,Distribution Channel,Duration (in seconds),...,Recipient Last Name,Recorded Date,"3. Current place of residence (State/Province,...",Response ID,Start Date,Response Type,User Language,Please write the email at which you would like...,Please indicate your preferred language to con...,Thank you for your interest in participating i...
2,30,,,,,,,,anonymous,716,...,,2022-12-12 16:25:13,Manresa,R_1IQPnZyzEWxzNWU,2022-12-12 16:13:16,IP Address,EN,sperry24@uwo.ca,,
3,69,,,,,,,,anonymous,159,...,,2023-01-20 22:34:52,California,R_12RVfAO1DLjA01T,2023-01-20 22:32:09,IP Address,EN,marthajack@gmail.com,,I am a monolingual speaker of English (I only ...
4,61,,,,,,,,anonymous,330,...,,2023-01-24 17:21:12,Texas,R_6zjRGPqZHCoPfwt,2023-01-24 17:15:40,IP Address,EN,latarver1@gmail.com,,I am a monolingual speaker of English (I only ...
5,62,,,,,,,,anonymous,172,...,,2023-01-25 04:38:10,Texas,R_3KrngfjPzN0tupu,2023-01-25 04:35:17,IP Address,EN,jmtarver1@comcast.net,,I am a monolingual speaker of English (I only ...


In [8]:
# only keep responses that were allowed to continue to BLP section
# ie speaker profiles of interest
df_new.Age.replace("",0, inplace = True)
df_new = df_new[df_new['Age']!=0].copy()
df_new.shape

(110, 104)

Remove irrelevant columns and rows with index 2 (pilot participant).

In [9]:
df_new.drop([2], axis=0, inplace = True)
df_new.reset_index(inplace = True, drop = True)
df_new.drop(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress', 'Duration (in seconds)',
         'RecordedDate', 'ResponseId', 'RecipientLastName', 'RecipientFirstName',
         'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude',
         'DistributionChannel', 'UserLanguage', 'Q115', 'Consent_intro_Id', 'Consent_intro_Name',
        'Consent_intro_Size', 'Consent_intro_Type', 'Q93_Id', 'Q93_Name', 'Q93_Size', 'Q93_Type',
            'Amazon_cat', 'Amazon_eng', 'Amazon_spa', 'Q120', 'Q121', 'Q122', 'Q122_4_TEXT',
            'lang_pref', 'email'],
        axis=1, inplace=True)
df_new.head()

,Age,Education,Education_6_TEXT,Finished,Gender,Q1#1_1,Q1#1_2,Q1#1_3,Q10#1_1,Q10#1_2,...,Q8#1_1,Q8#1_2,Q8#1_3,Q8#1_4,Q9#1_1,Q9#1_2,Q9#1_3,Q9#1_4,Residence,prof
0,1. Age1. EdadEdat1. Age,4. Highest level of education achieved - Selec...,4. Highest level of education achieved - Other...,Finished,2. Gender2. GéneroGènere2. Gender,5. At what age did you begin to learn the foll...,5. At what age did you begin to learn the foll...,5. At what age did you begin to learn the foll...,"14. When you talk to yourself, how often do yo...","14. When you talk to yourself, how often do yo...",...,"12. In an average week, what percentage of the...","12. In an average week, what percentage of the...","12. In an average week, what percentage of the...","12. In an average week, what percentage of the...","13. In an average week, what percentage of the...","13. In an average week, what percentage of the...","13. In an average week, what percentage of the...","13. In an average week, what percentage of the...","3. Current place of residence (State/Province,...",Thank you for your interest in participating i...
1,69,College,,True,Female,,,,,,...,,,,,,,,,California,I am a monolingual speaker of English (I only ...
2,61,Doctorate,,True,Female,,,,,,...,,,,,,,,,Texas,I am a monolingual speaker of English (I only ...
3,62,Doctorate,,True,Male,,,,,,...,,,,,,,,,Texas,I am a monolingual speaker of English (I only ...
4,40,College,,True,Male,,,,,,...,,,,,,,,,"California, United States",I am a monolingual speaker of English (I only ...


Rename partID column.

In [10]:
df_new.rename(columns={'Q33': 'partID'}, inplace = True)

In [11]:
# make all partIDs lowercase
df_new.partID = df_new.partID.map(str.lower)

Participant aoa302 and ama302 are the same, different names have been used mistakenly. Replace `aoa302` in the df with `ama302`.

In [12]:
df_new.partID = df_new.partID.str.replace('aoa302','ama302')

New column with mono, L1 eng, and L3 eng categories.

In [13]:
import numpy as np
prof = 'prof'
l1_eng = 'Q1#1_3'
conditions = [df_new[prof].str.startswith("I am"),
              (df_new[prof].str.startswith("I speak")) & ((df_new[l1_eng].str.startswith("Since")) | (df_new[l1_eng].str.startswith("Des"))), 
              (df_new[prof].str.startswith("I speak")) & ~((df_new[l1_eng].str.startswith("Since")) | (df_new[l1_eng].str.startswith("Des")))]
choices = ["mono", 'l1_eng', 'l3_eng']
df_new["lang_profile"] = np.select(conditions, choices, default=np.nan)

Fix column values that are written in multiple languages.

In [14]:
df_new.loc[df_new.Education.str.startswith("Men"), 'Education'] = "Less than high school"
df_new.loc[df_new.Education.str.startswith("Escola"), 'Education'] = "High school"
df_new.loc[df_new.Education.str.startswith("Uni"), 'Education'] = "College"
df_new.loc[df_new.Education.str.startswith("Doct"), 'Education'] = "Doctorate"
df_new.loc[df_new.Education == "Màster", 'Education'] = "Masters"
df_new.loc[df_new.Education == "Máster", 'Education'] = "Masters"
df_new.loc[df_new.Education == "Altres:", 'Education'] = "Other:"
df_new.loc[df_new.Education == "Otro:", 'Education'] = "Other:"
df_new.loc[df_new.Education.str.startswith("Form"), 'Education'] = "Professional training"

In [15]:
df_new.loc[df_new.Gender == "Mujer", 'Gender'] = "Female"
df_new.loc[df_new.Gender == "Dona", 'Gender'] = "Female"
df_new.loc[df_new.Gender.str.startswith("Hom"), 'Gender'] = "Male"

In [16]:
df_new.loc[df_new['Q1#1_1'].str.startswith("Des"), 'Q1#1_1'] = "Since birth"
df_new.loc[df_new['Q1#1_2'].str.startswith("Des"), 'Q1#1_2'] = "Since birth"
df_new.loc[df_new['Q1#1_3'].str.startswith("Des"), 'Q1#1_3'] = "Since birth"

In [17]:
df_new.loc[df_new['Q2#1_1'].str.startswith("Tan"), 'Q2#1_1'] = "As early as I can remember"
df_new.loc[df_new['Q2#1_2'].str.startswith("Tan"), 'Q2#1_2'] = "As early as I can remember"
df_new.loc[df_new['Q2#1_3'].str.startswith("Tan"), 'Q2#1_3'] = "As early as I can remember"

df_new.loc[df_new['Q2#1_1'] == "Encara no", 'Q2#1_1'] = "Still not comfortable"
df_new.loc[df_new['Q2#1_2'] == "Encara no", 'Q2#1_2'] = "Still not comfortable"
df_new.loc[df_new['Q2#1_3'] == "Encara no", 'Q2#1_3'] = "Still not comfortable"

df_new.loc[df_new['Q2#1_1'].str.startswith("Tod"), 'Q2#1_1'] = "Still not comfortable"
df_new.loc[df_new['Q2#1_2'].str.startswith("Tod"), 'Q2#1_2'] = "Still not comfortable"
df_new.loc[df_new['Q2#1_3'].str.startswith("Tod"), 'Q2#1_3'] = "Still not comfortable"

In [19]:
df_new.to_csv("data/blp_cleaned.csv", index = False)